# Generate Annotation Dataset
Script to convert text + label into ner format supported by GliNER or Flair

In [1]:
FOLDER = "microsoft-learn-location-mention-recognition-challenge20240905-10153-193u9hv" # update this path https://zindi.africa/competitions/microsoft-learn-location-mention-recognition-challenge/data

import pandas as pd
import os
import json
import numpy as np

os.makedirs("data/accepted_data", exist_ok=True)

train = pd.read_csv(os.path.join(FOLDER, "Train_1.csv"))

train.sample(10)

,tweet_id,text,location
63862,ID_909433914506215424,RT @FLAFLCIO: Visit our new website to apply f...,NaN
6305,ID_1032287088190808064,NaN,Kerala
70452,ID_913625501440380928,NaN,United States
25148,ID_1109454892576452608,"As the sun sets, a 2nd truck is loaded and rea...",NaN
56714,ID_906167049931632640,NaN,Florida
72742,ID_914857829684695040,NaN,Puerto Rico U.S.
7019,ID_1032637816096808960,NaN,Kerala
37160,ID_1176567269553557504,NaN,Pakistan
27470,ID_1112343800700121088,NaN,NaN
5711,ID_1032133583165513728,NaN,NaN


In [2]:
len(train)

73072

In [3]:
train.isna().sum()

tweet_id        0
text        56624
location    29612
dtype: int64

In [4]:
train = train.dropna(subset=["text", "location"], how="all")

len(train)

48059

In [5]:
train.isna().sum()

tweet_id        0
text        31611
location     4599
dtype: int64

Text can't be Nan so dropped

In [6]:
train = train.dropna(subset=["text"], how="all")

len(train)

16448

In [8]:
train.sample(10)

,tweet_id,text,location
17903,ID_1062043965636132864,As three record-breaking wild fires continue t...,California
37119,ID_1176558963938254848,4/4 Doctors and paramedics are providing neces...,NaN
43773,ID_784649154194710528,"Flash Flood Warning for Bamberg, Calhoun, Clar...",Bamberg Calhoun Clarendon Orangeburg Richland ...
71527,ID_914176142327914496,Now this is something I can trust. DONATE!!!,NaN
14981,ID_1041985289210785792,Death toll so far from #HurricaneFlorence 34Tr...,Puerto Rico
69724,ID_913251069802692608,Hurricane Maria : Charity Navigator list of ch...,NaN
40724,ID_732607952192835584,Explosion now confirmed at a home in Fort McMu...,Fort McMurray
27103,ID_1111689287857332224,OMAHA! Thank you for coming out last night. Ju...,Nebraska OMAHA
26167,ID_1110802600180568064,"While you go about your day, remember those wh...",NaN
42278,ID_771458098426966016,@gillhp rich got woken up by it | tsunami war...,NaN


In [11]:
train.iloc[0]

tweet_id                               ID_1001136696589631488
text        Flash floods struck a Maryland city on Sunday,...
location                                             Maryland
Name: 1, dtype: object

In [12]:
train.iloc[0]["text"]

'Flash floods struck a Maryland city on Sunday, washing out streets and tossing cars like bath toys.'

In [13]:
# Huge discussion with GPT to finally came out with this code
import re

def split_and_clean(sentence):
	# Split sentence into words while keeping punctuation as separate tokens
	words = re.findall(r"[\w]+|[^\s\w]", sentence)
	return words

def normalize_word(word):
	# Remove leading # and @ symbols for comparison purposes
	return word.lstrip("#@")

def annotate_words(sentence_a, sentence_b):
	# Split both sentences into lists of words
	words_a = split_and_clean(sentence_a)
	words_b = split_and_clean(sentence_b)
	
	# Create a set of normalized words and phrases from B
	phrases_b = set()
	i = 0
	while i < len(words_b):
		# Add both individual words and 2-word phrases for matching
		phrases_b.add(normalize_word(words_b[i]))
		if i < len(words_b) - 1:
			phrases_b.add(f"{normalize_word(words_b[i])} {normalize_word(words_b[i+1])}")
		i += 1

	annotated_words = []
	i = 0

	while i < len(words_a):
		# Check if current and next word in A match a 2-word phrase in B
		if i < len(words_a) - 1:
			current_phrase = f"{normalize_word(words_a[i])} {normalize_word(words_a[i+1])}"
			if current_phrase in phrases_b:
				annotated_words.append((words_a[i], True))  # Mark first word of phrase as True
				annotated_words.append((words_a[i+1], True))  # Mark second word of phrase as True
				i += 2  # Skip to next after the phrase
				continue
		
		# Check individual word
		annotated_words.append((words_a[i], normalize_word(words_a[i]) in phrases_b))
		i += 1
	
	return annotated_words

# Example sentences
A = "Watch Live City: Aerials of damage after historic flash flooding in Ellicott City, Maryland."
B = "Ellicott City Maryland"

# Annotate words in A
annotations = annotate_words(A, B)

# Print annotated words
for word, is_in_b in annotations:
	print(f"{word}: {is_in_b}")

Watch: False
Live: False
City: True
:: False
Aerials: False
of: False
damage: False
after: False
historic: False
flash: False
flooding: False
in: False
Ellicott: True
City: True
,: False
Maryland: True
.: False


In [15]:
import re

def split_and_clean(sentence):
	# Split sentence into words while keeping punctuation as separate tokens
	words = re.findall(r"[\w]+|[^\s\w]", sentence)
	return words

def normalize_word(word):
	# Remove leading # and @ symbols for comparison purposes
	return word.lstrip("#@")

def annotate_words(sentence_a, sentence_b):
	# Split both sentences into lists of words
	words_a = split_and_clean(sentence_a)
	words_b = split_and_clean(sentence_b)
	
	# Create a set of normalized words and 2-word phrases from B
	phrases_b = set()
	i = 0
	while i < len(words_b):
		# Add both individual words and 2-word phrases for matching
		phrases_b.add(normalize_word(words_b[i]))
		if i < len(words_b) - 1:
			phrases_b.add(f"{normalize_word(words_b[i])} {normalize_word(words_b[i+1])}")
		i += 1

	annotated_words = []
	i = 0

	while i < len(words_a):
		# Check if current and next word in A match a 2-word phrase in B
		if i < len(words_a) - 1:
			current_phrase = f"{normalize_word(words_a[i])} {normalize_word(words_a[i+1])}"
			if current_phrase in phrases_b:
				annotated_words.append((words_a[i], True))  # Mark first word of phrase as True
				annotated_words.append((words_a[i+1], True))  # Mark second word of phrase as True
				i += 2  # Skip to next after the phrase
				continue
		
		# Check individual word (only if it's not part of a multi-word match)
		annotated_words.append((words_a[i], normalize_word(words_a[i]) in phrases_b))
		i += 1
	
	return annotated_words

# Example sentences
A = "Watch Live City: Aerials of damage after historic flash flooding in Ellicott City, Maryland."
B = "Ellicott City Maryland"

# Annotate words in A
annotations = annotate_words(A, B)

# Print annotated words
for word, is_in_b in annotations:
	print(f"{word}: {is_in_b}")


Watch: False
Live: False
City: True
:: False
Aerials: False
of: False
damage: False
after: False
historic: False
flash: False
flooding: False
in: False
Ellicott: True
City: True
,: False
Maryland: True
.: False


In [17]:
import re

def split_and_clean(sentence):
	# Split sentence into words while keeping punctuation as separate tokens
	words = re.findall(r"[\w]+|[^\s\w]", sentence)
	return words

def normalize_word(word):
	# Remove leading # and @ symbols for comparison purposes
	return word.lstrip("#@")

def find_multiword_phrases(words_a, words_b):
	# Find all possible multi-word sequences in B that are present in A
	phrases_b = []
	len_b = len(words_b)

	for start in range(len_b):
		for end in range(start + 1, len_b + 1):
			phrase = " ".join(normalize_word(word) for word in words_b[start:end])
			if phrase in " ".join(normalize_word(word) for word in words_a):
				phrases_b.append(phrase)
	
	return phrases_b

def annotate_words(sentence_a, sentence_b):
	# Split both sentences into lists of words
	words_a = split_and_clean(sentence_a)
	words_b = split_and_clean(sentence_b)
	
	# Find all multi-word phrases in B that are in A
	multiword_phrases_b = find_multiword_phrases(words_a, words_b)
	
	# Annotate words in A
	annotated_words = []
	i = 0
	
	while i < len(words_a):
		# Check if the current word starts a multi-word phrase in B
		found_match = False
		for phrase in multiword_phrases_b:
			phrase_words = phrase.split()
			phrase_len = len(phrase_words)
			
			# Check if the next `phrase_len` words in A match the phrase
			if words_a[i:i + phrase_len] == phrase_words:
				# Mark the entire phrase as True
				for j in range(phrase_len):
					annotated_words.append((words_a[i + j], True))
				i += phrase_len  # Skip the words in the phrase
				found_match = True
				break
		
		if not found_match:
			# If no phrase is found, annotate the word normally
			annotated_words.append((words_a[i], normalize_word(words_a[i]) in words_b))
			i += 1
	
	return annotated_words

# Example sentences
A = "Watch Live City: Aerials of damage after historic flash flooding in Ellicott City, Maryland."
B = "Ellicott City Maryland"

# Annotate words in A
annotations = annotate_words(A, B)

# Print annotated words
for word, is_in_b in annotations:
	print(f"{word}: {is_in_b}")


Watch: False
Live: False
City: True
:: False
Aerials: False
of: False
damage: False
after: False
historic: False
flash: False
flooding: False
in: False
Ellicott: True
City: True
,: False
Maryland: True
.: False


In [19]:
import re

def split_and_clean(sentence):
	# Split sentence into words while keeping punctuation as separate tokens
	words = re.findall(r"[\w]+|[^\s\w]", sentence)
	return words

def normalize_word(word):
	# Remove leading # and @ symbols for comparison purposes
	return word.lstrip("#@")

def find_multiword_phrases(words_a, words_b):
	phrases_b = []
	len_a = len(words_a)
	len_b = len(words_b)
	
	# Join words_a and words_b into normalized space-separated strings
	normalized_a = " ".join(normalize_word(word) for word in words_a)
	
	# Start from any word in B and create multi-word phrases
	for start in range(len_b):
		for end in range(start + 1, len_b + 1):
			# Form a phrase from words_b[start:end]
			phrase = " ".join(normalize_word(word) for word in words_b[start:end])
			if phrase in normalized_a:
				phrases_b.append(phrase)
	
	return phrases_b


def annotate_words(sentence_a, sentence_b):
	# Split both sentences into lists of words
	words_a = split_and_clean(sentence_a)
	words_b = split_and_clean(sentence_b)
	
	# Find all multi-word phrases in B that are in A
	multiword_phrases_b = find_multiword_phrases(words_a, words_b)
	
	# Annotate words in A
	annotated_words = []
	i = 0
	
	while i < len(words_a):
		# Check if the current word starts a multi-word phrase in B
		found_match = False
		for phrase in multiword_phrases_b:
			phrase_words = phrase.split()
			phrase_len = len(phrase_words)
			
			# Check if the next `phrase_len` words in A match the phrase
			if words_a[i:i + phrase_len] == phrase_words:
				# Mark the entire phrase as True
				for j in range(phrase_len):
					annotated_words.append((words_a[i + j], True))
				i += phrase_len  # Skip the words in the phrase
				found_match = True
				break
		
		if not found_match:
			# If no phrase is found, annotate the word normally
			annotated_words.append((words_a[i], normalize_word(words_a[i]) in words_b))
			i += 1
	
	return annotated_words

# Example sentences
A = "Watch Live City: Aerials of damage after historic flash flooding in Ellicott City, Maryland."
B = "Ellicott City Maryland"

# Annotate words in A
annotations = annotate_words(A, B)

# Print annotated words
for word, is_in_b in annotations:
	print(f"{word}: {is_in_b}")


Watch: False
Live: False
City: True
:: False
Aerials: False
of: False
damage: False
after: False
historic: False
flash: False
flooding: False
in: False
Ellicott: True
City: True
,: False
Maryland: True
.: False


In [24]:
def find_multiword_phrases(words_a, words_b):
	phrases_b = []
	len_b = len(words_b)
	
	# Join words_a into a normalized space-separated string
	normalized_a = " ".join(normalize_word(word) for word in words_a)
	
	i = 0
	while i < len_b:
		# Try to form the longest possible phrase from words_b that exists in words_a
		for end in range(len_b, i, -1):  # Start checking from the longest possible phrase
			phrase = " ".join(normalize_word(word) for word in words_b[i:end])
			if phrase in normalized_a:
				phrases_b.append(phrase)  # Add the phrase
				i = end - 1  # Skip the words in the phrase
				break
		i += 1
	
	return phrases_b

# Example input sentences
words_a = split_and_clean("Watch Live City: Aerials of damage after historic flash flooding Love in Ellicott City, Maryland.")
words_b = split_and_clean("Ellicott City Love Maryland")

# Call find_multiword_phrases
phrases = find_multiword_phrases(words_a, words_b)

# Output the found phrases
print(phrases)

['Ellicott City', 'Love', 'Maryland']


In [25]:
# Example sentences
A = "Watch Live City: Aerials of damage after historic flash flooding in Ellicott City, Maryland."
B = "Ellicott City Maryland"

# Annotate words in A
annotations = annotate_words(A, B)

# Print annotated words
for word, is_in_b in annotations:
	print(f"{word}: {is_in_b}")

Watch: False
Live: False
City: True
:: False
Aerials: False
of: False
damage: False
after: False
historic: False
flash: False
flooding: False
in: False
Ellicott: True
City: True
,: False
Maryland: True
.: False


In [41]:
import re


def split_and_clean(sentence):
    # Split sentence into words while keeping punctuation as separate tokens
    words = re.findall(r"[\w]+|[^\s\w]", sentence)
    return words


def normalize_word(word):
    # Remove leading # and @ symbols for comparison purposes
    return word.lstrip("#@")


def find_multiword_phrases(words_a, words_b) -> list[str]:
    phrases_b = []
    len_b = len(words_b)

    # Join words_a into a normalized space-separated string
    normalized_a = " ".join(normalize_word(word) for word in words_a)

    i = 0
    while i < len_b:
        # Try to form the longest possible phrase from words_b that exists in words_a
        for end in range(
            len_b, i, -1
        ):  # Start checking from the longest possible phrase
            phrase = " ".join(normalize_word(word) for word in words_b[i:end])
            if phrase in normalized_a:
                phrases_b.append(phrase)  # Add the phrase
                i = end - 1  # Skip the words in the phrase
                break
        i += 1

    return phrases_b


def annotate_words(sentence_a, sentence_b):
    # Split both sentences into lists of words
    words_a = split_and_clean(sentence_a)
    words_b = split_and_clean(sentence_b)

    # Find all multi-word phrases in B that are in A
    words_b = find_multiword_phrases(words_a, words_b)

    # Annotate words in A
    annotated_words = []
    n_ignored = set()
    for i, word in enumerate(words_a):
        if i in n_ignored:
            continue
        possible_match = [w for w in words_b if w.startswith(word)]
        if not possible_match:
            annotated_words.append((word, "O"))
            continue
        decision = [
            " ".join(words_a[i : i + len(w.split())]) == w for w in possible_match
        ]
        if any(decision):
            dec, w = sorted(
                zip(decision, possible_match), key=lambda x: (-x[0], -len(x[1].split()))
            )[0]
            n_ignored.update([n for n in range(i, i + len(w.split()))])

            annotated_words.extend(
                [(w, "B-LOC" if k == 0 else "I-LOC") for k, w in enumerate(words_a[i : i + len(w.split())])]
            )
        else: # Fake starting match !
            annotated_words.append((word, "O"))

    return annotated_words


# Example sentences
A = "Watch Live City: Aerials of damage after historic flash flooding in City Ellicott, Maryland."
B = "City Ellicott Maryland"

# Annotate words in A
annotations = annotate_words(A, B)

# Print annotated words
for word, is_in_b in annotations:
    print(f"{word}: {is_in_b}")

Watch: O
Live: O
City: O
:: O
Aerials: O
of: O
damage: O
after: O
historic: O
flash: O
flooding: O
in: O
City: B-LOC
Ellicott: I-LOC
,: O
Maryland: B-LOC
.: O


In [42]:

# Example sentences
A = "Watch Live City: Aerials of damage after historic flash flooding in City Ellicott, Maryland."
B = ""

# Annotate words in A
annotations = annotate_words(A, B)

# Print annotated words
for word, is_in_b in annotations:
    print(f"{word}: {is_in_b}")

Watch: O
Live: O
City: O
:: O
Aerials: O
of: O
damage: O
after: O
historic: O
flash: O
flooding: O
in: O
City: O
Ellicott: O
,: O
Maryland: O
.: O


In [35]:
train.isna().sum()

tweet_id       0
text           0
location    4599
dtype: int64

In [36]:
train.fillna("", inplace=True)

train.sample(10)

,tweet_id,text,location
36637,ID_1176505873419689984,No damages have been reported so far in #Kashm...,Kashmir Valley Mirpur
34825,ID_1168308758998671360,JUST IN: I-26 lane reversal starting at NOON o...,
39355,ID_728667676155879424,Our <3 goes out to those affected by Fort McMu...,McMurray
36075,ID_1176473682216202240,Yah ALLAH reham ! Karachi damaged by the 10 mi...,Pakistan
41539,ID_768563495629717504,• #ItalyEarthquake • The highest number of vic...,Amatrice Italy Rieti
48462,ID_902637254660812800,#Rockport and #AransasCounty need your help! T...,AransasCounty Houston
20646,ID_1064659382607110144,Most local governments dealing with California...,California
45723,ID_870152565601828864,RT @AmanthaP: APPEAL @SLRedCross running low o...,Galle Gampaha Matara
8974,ID_1035336737868767232,RT @Anulalindia: Water rose to the level of tw...,
1263,ID_1022095810014703616,People at Thriasio hospital waiting to donate ...,Thriasio hospital


In [38]:
train.loc[41539, "text"]

'• #ItalyEarthquake • The highest number of victims in #Rieti. #Amatrice is actually a ghost town. Death toll: 159. #earthquake #Italy'

In [43]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

thr = ThreadPoolExecutor()
results = list(
    tqdm(
        thr.map(annotate_words, train["text"].values, train["location"].values),
        total=len(train),
    )
)

100%|██████████| 16448/16448 [00:00<00:00, 87838.04it/s]


In [44]:
results[0]

[('Flash', 'O'),
 ('floods', 'O'),
 ('struck', 'O'),
 ('a', 'O'),
 ('Maryland', 'B-LOC'),
 ('city', 'O'),
 ('on', 'O'),
 ('Sunday', 'O'),
 (',', 'O'),
 ('washing', 'O'),
 ('out', 'O'),
 ('streets', 'O'),
 ('and', 'O'),
 ('tossing', 'O'),
 ('cars', 'O'),
 ('like', 'O'),
 ('bath', 'O'),
 ('toys', 'O'),
 ('.', 'O')]

In [45]:
train.iloc[0]

tweet_id                               ID_1001136696589631488
text        Flash floods struck a Maryland city on Sunday,...
location                                             Maryland
Name: 1, dtype: object

In [48]:
def annotate_words(sentence_a, sentence_b):
    # Split both sentences into lists of words
    words_a = split_and_clean(sentence_a)
    words_b = split_and_clean(sentence_b)

    # Find all multi-word phrases in B that are in A
    words_b = find_multiword_phrases(words_a, words_b)

    # Annotate words in A
    annotated_words = []
    n_ignored = set()
    entities = []
    for i, word in enumerate(words_a):
        if i in n_ignored:
            continue
        possible_match = [w for w in words_b if w.startswith(word)]
        if not possible_match:
            annotated_words.append((word, "O"))
            continue
        decision = [
            " ".join(words_a[i : i + len(w.split())]) == w for w in possible_match
        ]
        if any(decision):
            dec, w = sorted(
                zip(decision, possible_match), key=lambda x: (-x[0], -len(x[1].split()))
            )[0]
            n_ignored.update([n for n in range(i, i + len(w.split()))])

            entities.append([i, i - 1 + len(w.split())])

            annotated_words.extend(
                [
                    (w, "B-LOC" if k == 0 else "I-LOC")
                    for k, w in enumerate(words_a[i : i + len(w.split())])
                ]
            )
        else:  # Fake starting match !
            annotated_words.append((word, "O"))

    return annotated_words, entities

In [51]:
# Example sentences
A = "Watch Live City: Aerials of damage after historic flash flooding in City Ellicott, Maryland."
B = "City Ellicott Maryland"

# Annotate words in A
annotate_words(A, B)

([('Watch', 'O'),
  ('Live', 'O'),
  ('City', 'O'),
  (':', 'O'),
  ('Aerials', 'O'),
  ('of', 'O'),
  ('damage', 'O'),
  ('after', 'O'),
  ('historic', 'O'),
  ('flash', 'O'),
  ('flooding', 'O'),
  ('in', 'O'),
  ('City', 'B-LOC'),
  ('Ellicott', 'I-LOC'),
  (',', 'O'),
  ('Maryland', 'B-LOC'),
  ('.', 'O')],
 [[12, 13], [15, 15]])

In [52]:
results_with_annotation = list(
    tqdm(
        thr.map(annotate_words, train["text"].values, train["location"].values),
        total=len(train),
    )
)

100%|██████████| 16448/16448 [00:00<00:00, 75682.56it/s]


In [53]:
results_with_annotation[10]

([('Howard', 'B-LOC'),
  ('County', 'O'),
  ('Executive', 'O'),
  ('Allan', 'O'),
  ('Kittleman', 'O'),
  ('said', 'O'),
  ('Monday', 'O'),
  ('morning', 'O'),
  ('that', 'O'),
  ('his', 'O'),
  ('priorities', 'O'),
  ('are', 'O'),
  ('finding', 'O'),
  ('the', 'O'),
  ('missing', 'O'),
  ('man', 'O'),
  ('and', 'O'),
  ('assessing', 'O'),
  ('the', 'O'),
  ('condition', 'O'),
  ('of', 'O'),
  ('buildings', 'O'),
  ('that', 'O'),
  ('house', 'O'),
  ('shops', 'O'),
  (',', 'O'),
  ('restaurants', 'O'),
  ('and', 'O'),
  ('families', 'O'),
  ('.', 'O')],
 [[0, 0]])

In [54]:
lines = [
    "\n".join(f"{w}\t{lab}" for w, lab in preds) for preds, _ in results_with_annotation
]

text_train = "\n\n".join(lines)
len(text_train)

3329539

In [55]:
with open("data/accepted_data/FullTrainCleaned.txt", "w") as file:
    file.write(text_train)

In [58]:
lines_annotated = [
    {
        "tokenized_text": [w for w, _ in preds],
        "ner": [tag + ["location"] for tag in tags],
    } for preds, tags in results_with_annotation
]

lines_annotated[0]

{'tokenized_text': ['Flash',
  'floods',
  'struck',
  'a',
  'Maryland',
  'city',
  'on',
  'Sunday',
  ',',
  'washing',
  'out',
  'streets',
  'and',
  'tossing',
  'cars',
  'like',
  'bath',
  'toys',
  '.'],
 'ner': [[4, 4, 'location']]}

In [59]:
with open("data/accepted_data/FullTrainCleaned.json", "w") as file:
    json.dump(lines_annotated, file, indent=4)

In [61]:
location_count = pd.DataFrame([
    len(i["ner"]) for i in lines_annotated
], columns=["NLabel"])

location_count.describe()

,NLabel
count,16448.000000
mean,1.068033
std,1.015442
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,17.000000


In [63]:
location_count.value_counts()

NLabel
1         8058
0         4608
2         2570
3          797
4          254
5           82
6           42
7           22
8            6
9            4
12           2
10           1
11           1
17           1
Name: count, dtype: int64

In [64]:
location_count.loc[location_count["NLabel"] > 5, "NLabel"] = 6

location_count.value_counts()

NLabel
1         8058
0         4608
2         2570
3          797
4          254
5           82
6           79
Name: count, dtype: int64

In [65]:
from sklearn.model_selection import train_test_split

train_index, test_index = train_test_split(
    location_count, test_size=.1, stratify=location_count["NLabel"].values
)

len(train_index), len(test_index)

(14803, 1645)

In [66]:
train_index["NLabel"].value_counts()

NLabel
1    7252
0    4147
2    2313
3     717
4     229
5      74
6      71
Name: count, dtype: int64

In [67]:
test_index["NLabel"].value_counts()

NLabel
1    806
0    461
2    257
3     80
4     25
6      8
5      8
Name: count, dtype: int64

In [68]:
train_lines_annotated = [
    lines_annotated[i] for i in train_index.index
]

test_lines_annotated = [
    lines_annotated[i] for i in test_index.index
]

In [69]:
os.makedirs("data/accepted_data", exist_ok=True)

with open("data/accepted_data/TrainCleaned.json", "w") as file:
    json.dump(train_lines_annotated, file, indent=4)

with open("data/accepted_data/TestCleaned.json", "w") as file:
    json.dump(test_lines_annotated, file, indent=4)

In [70]:
train_lines = [
    lines[i] for i in train_index.index
]

test_lines = [
    lines[i] for i in test_index.index
]

In [72]:
os.makedirs("data/accepted_data", exist_ok=True)

with open("data/accepted_data/TrainCleaned.txt", "w") as file:
    file.write("\n\n".join(train_lines), )

with open("data/accepted_data/TestCleaned.txt", "w") as file:
    file.write("\n\n".join(test_lines), )